Arthur Catarino de Oliveira  
Guilherme Fidélis Freire  
Mateus Poddis Correa

##Dados
- A: Alimentos que podem ser incluídos na dieta.
- N: Nutrientes que possuem um máximo desejável na dieta(Gordura e Sódio).
- Nmaxj: Limite maximo do nutriente j, j E N
- Yij: valor referente ao nutriente j do alimento i, j E N e i E A
- Cmax: Custo máximo da dieta em reais.
- Kmax: Máximo de kilocalorias na dieta.

- minProt: Quantidade mímima de proteína em gramas na dieta.
- minCarb: Quantidade mímima de carboidrato em gramas na dieta.
- minCal: Quantidade mímima de cálcio em miligramas na dieta.
- minFe: Quantidade mímima de ferro em miligramas na dieta.
- minVitc: Quantidade mímima de vitamina C em miligramas na dieta.
- fatMax: Quantidade máxima desejada de gordura em gramas na dieta.
- sodMax: Quantidade máxima desejada de sódio em miligramas na dieta.
- precoi, i E A: Preço em reais da porção do alimento i.
- Kcali, i E A: Quantidade de kilocalorias da porção do alimento i.
- proti, i E A: Quantidade em gramas de proteína na porção do alimento i.
- fati, i E A: Quantidade em gramas de gordura na porção do alimento i.
- carbi, i E A: Quantidade em gramas de carboidrato na porção do alimento i.
- cali, i E A: Quantidade em miligramas de cálcio na porção do alimento i.
- fei, i E A: Quantidade em miligramas de ferro na porção do alimento i.
- vitCi, i E A: Quantidade em miligramas de vitamina C na porção do alimento i.
- sodi, i E A: Quantidade em miligramas de sódio na porção do alimento i.

##Variaveis de decisão
- Qi, i E A, quantidade de porções do alimento i.

##Função Objetivo:
$$
Minimizar: \sum_{j E N}^{}-NutMax_j \sum_{i E A}^{}Q_i*Y_{ji}
$$

##Restrições:
1) Restrição que respeita o preço diário máximo da dieta.
$$
 \sum_{i E A}^{} Q_i*preco_i <= Cmax
$$
2) Restrição que respeita o máximo de kilocalorias da dieta.
$$
 \sum_{i E A}^{} Q_i*Kcal_i <= Kmax
$$
3) Restrição que garante o mínimo de proteína na dieta.
$$
 \sum_{i E A}^{} Q_i*prot_i >= minProt
$$
4) Restrição que garante o mínimo de carboidrato na dieta.
$$
 \sum_{i E A}^{} Q_i*carb_i >= minCarb
$$
5) Restrição que garante o mínimo de cálcio na dieta.
$$
 \sum_{i E A}^{} Q_i*cal_i >= minCal
$$

6) Restrição que garante o mínimo de ferro na dieta.
$$
 \sum_{i E A}^{} Q_i*fe_i >= minFe
$$
7) Restrição que garante o mínimo de vitamina C na dieta.
$$
 \sum_{i E A}^{} Q_i*vitC_i >= minVitC
$$
10) Restrição de não negatividade.
$Q_i >= 0$ $∀ i E A$

$X_j >= 0$ $∀ j E N$

In [ ]:
!pip install pulp
import pulp

Para esse problema é criado um classe Alimento para pegar os dados de dieta.json. E eles são armazenados em um dicionário (id, Alimento). Além disso são criadas variáveis para pegar os parametros também. Obs: Depois de rodar uma vez e escolher o arquivo dieta.json, a linha "uploaded = files.upload()" pode ser excluída.

In [ ]:
from google.colab import files
import json

uploaded = files.upload()

class Alimento:
    def __init__(self, id, nome, preco, kcal, proteina, gordura, carboidrato,
                 calcio, ferro, vitaminaC, sodio, porcao):
        self.id = id
        self.nome = nome
        self.preco = preco
        self.kcal = kcal
        self.proteina = proteina
        self.gordura = gordura
        self.carboidrato = carboidrato
        self.calcio = calcio
        self.ferro = ferro
        self.vitaminaC = vitaminaC
        self.sodio = sodio
        self.porcao = porcao

with open("dieta.json", "r", encoding="utf-8") as f:
    data = json.load(f)

params = data["params"]
Kmax = params["Kmax"]
Cmax = params["Cmax"]
minProt = params["min_prot"]
minCarb = params["min_carb"]
minCal = params["min_cal"]
minFe = params["min_fe"]
minVitc = params["min_vitc"]
fatMax = params["fat_max"]*1000
sodMax = params["sod_max"]

alimentos = {
    item["id"]: Alimento(**item) for item in data["alimentos"]
}

for a in alimentos:
  alimentos[a].gordura = alimentos[a].gordura*1000

NutrientesComRestrição = {"gordura":50*1000,
                          "sodio":2300}

Inicializando o modelo

In [ ]:
modelo = pulp.LpProblem('Dieta', pulp.LpMinimize)
quantidadePorcao = pulp.LpVariable.dicts('quantidade porção', list(alimentos.keys()), lowBound=0)

In [ ]:
restricoes = []
for n in NutrientesComRestrição.keys():
    expressao = pulp.lpSum([quantidadePorcao[i] * getattr(alimentos[i], n) for i in alimentos])
    restricoes.append(expressao - NutrientesComRestrição[n])

modelo += pulp.lpSum(restricoes)

##Função Objetivo:
$$
Minimizar: \sum_{j E N}^{}-NutMax_j \sum_{i E A}^{}Q_i*Y_{ji}
$$

In [ ]:
restricoes = []
for n in NutrientesComRestrição.keys():
    expressao = pulp.lpSum([quantidadePorcao[i] * getattr(alimentos[i], n) for i in alimentos])
    restricoes.append(expressao - NutrientesComRestrição[n])

modelo += pulp.lpSum(restricoes)

Restrições:

1) $\sum_{i E A}^{} Q_i*preco_i$ <= Cmax   (Custo máximo).


In [ ]:
modelo += pulp.lpSum(quantidadePorcao[i] * alimentos[i].preco for i in alimentos) <= Cmax

2) $\sum_{i E A}^{} Q_i*Kcal_i$ <= Kmax (Máximo de kcal).

In [ ]:
modelo += pulp.lpSum(quantidadePorcao[i] * alimentos[i].kcal for i in alimentos) <= Kmax

3) $\sum_{i E A}^{} Q_i*prot_i$ >= minProt (Mínimo de proteína).

In [ ]:
modelo += pulp.lpSum(quantidadePorcao[i] * alimentos[i].proteina for i in alimentos) >= minProt

4) $\sum_{i E A}^{} Q_i*carb_i$ >= minCarb (Mínimo de carboidrato).

In [ ]:
modelo += pulp.lpSum(quantidadePorcao[i] * alimentos[i].carboidrato for i in alimentos) >= minCarb

5) $\sum_{i E A}^{} Q_i*cal_i$ >= minCal (Mínimo de cálcio).

In [ ]:
modelo += pulp.lpSum(quantidadePorcao[i] * alimentos[i].calcio for i in alimentos) >= minCal

6) $\sum_{i E A}^{} Q_i*fe_i$ >= minFe (Mínimo de ferro).

In [ ]:
modelo += pulp.lpSum(quantidadePorcao[i] * alimentos[i].ferro for i in alimentos) >= minFe

7) $\sum_{i E A}^{} Q_i*vitC_i$ >= minVitC (Mínimo de vitamina C).

In [ ]:
modelo += pulp.lpSum(quantidadePorcao[i] * alimentos[i].vitaminaC for i in alimentos) >= minVitc

8) Restrição de não negatividade.

$Q_i >= 0$ $∀ i E A$

$X_j >= 0$ $∀ j E N$

In [ ]:
print(modelo)

Dieta:
MINIMIZE
1000.0*quantidade_porção_1 + 10000.0*quantidade_porção_10 + 110.0*quantidade_porção_11 + 1500.0*quantidade_porção_12 + 215.0*quantidade_porção_13 + 105.0*quantidade_porção_14 + 206.0*quantidade_porção_15 + 560.0*quantidade_porção_16 + 310.0*quantidade_porção_17 + 208.0*quantidade_porção_18 + 100.0*quantidade_porção_19 + 501.0*quantidade_porção_2 + 300.0*quantidade_porção_20 + 15010.0*quantidade_porção_21 + 18002.0*quantidade_porção_22 + 16000.0*quantidade_porção_23 + 500.0*quantidade_porção_24 + 401.0*quantidade_porção_25 + 3000.0*quantidade_porção_26 + 500.0*quantidade_porção_27 + 10000.0*quantidade_porção_28 + 9080.0*quantidade_porção_29 + 2000.0*quantidade_porção_3 + 3030.0*quantidade_porção_30 + 3500.0*quantidade_porção_4 + 5000.0*quantidade_porção_5 + 3002.0*quantidade_porção_6 + 7000.0*quantidade_porção_7 + 15000.0*quantidade_porção_8 + 2500.0*quantidade_porção_9 + -52300.0
SUBJECT TO
_C1: 0.5 quantidade_porção_1 + 1.8 quantidade_porção_10
 + 0.8 quantidade_porção

Solução do problema.

In [ ]:
status = modelo.solve()

print(f'Status: {pulp.LpStatus[status]}')
print(f'Função objetivo: {pulp.value(modelo.objective)}')
for i in alimentos:
    print(f'Q{i}: {pulp.value(quantidadePorcao[i])}')

Status: Optimal
Função objetivo: -48388.5857166
Q1: 0.0
Q2: 0.0
Q3: 0.0
Q4: 0.0
Q5: 0.0
Q6: 0.0
Q7: 0.0
Q8: 0.0
Q9: 0.0
Q10: 0.0
Q11: 0.0
Q12: 0.0
Q13: 0.0
Q14: 7.3876872
Q15: 0.0
Q16: 0.0
Q17: 0.0
Q18: 0.0
Q19: 14.276206
Q20: 0.0
Q21: 0.0
Q22: 0.0
Q23: 0.0
Q24: 0.0
Q25: 4.2595674
Q26: 0.0
Q27: 0.0
Q28: 0.0
Q29: 0.0
Q30: 0.0
